### Ćelijski automati

Ćelijski automati su klasa modela koji se mogu koristiti za simulaciju različitih prirodnih fenomena. Korišćeni su za modeliranje razvoja tumora, kroz trodimenzionalnu mapu tkiva i predviđanje rasta ćelija kroz vreme. U neuronauci, ćelijski automati se mogu koristi za izučavanje populacije aktivacije neurona. Postoje mnogo druge primene, uključujući one u hemiji, biologiji, ekologiji i mnogim drugim granama nauke.

Ćelijski automati su u osnovi skup (prostorno) uređenih jedinica koje nazivamo ćelije. Svaka ćelija vrši neku od mogućih akcija, ili uzima neko od mogućih stanja, u zavisnosti od svoje neposredne okoline. Možda najpoznatiji primer ćelijskih automata je Konvejeva Igra života.

### Igra života

Konvejeva Igra života je ćelijski automat koji se sastoji iz pravougaone mreže ćelija. Svaka ćeilja može biti u jednom od sva stanja: *živa* i *mrtva*. Igra se odvija kroz vremenske iteracije, gde ćelije u jednoj iteraciji računaju svoje stanje u sledećoj. Stanje ćelije se računa na osnovu svog trenutnog stanja, trenutnog stanja svih neposrednih 8 suseda ćelije i sledećih pravila:
- Ako ćelija ima manje od 2 živa suseda, u sledećoj iteraciji će biti mrtva 
- Ako ćelija ima više od 3 živa suseda, u sledećoj iteraciji će biti mrtva 
- Ako je ćelija živa i ima 2 ili 3 živa suseda, u sledećoj iteraciji će biti živa
- Ako je ćelija mrtva i ima 3 živa suseda, u sledećoj iteraciji će biti živa

Osnovna struktura podataka Igre života je matrica stanja, koja za svaku ćeliju a[i, j] sadrži podatak da li je ćelija živa ili mrtva. Pri implementaciji igre javlja se problem suseda ćelija koje se nalaze uz ivicu mreže (ćelije u prvoj koloni nemaju "leve" susede, itd.). Postoji više rešenja za ovaj problem kao što su:
- Pretpostaviti da su nepostojeći susedi uvek mrtvi
- Uvesti "cikluse", tako da je poslednja kolona "levi" sused prve kolone, a poslednji red "gornji" sused prvog reda (efektivno torus).

Pri izradi zadataka koristiti drugi pristup.

### Zadatak
Zadatak je implementirati paralelizovanu verziju Igre života u programskom jeziku Python, na nekoliko načina. Prilikom implementacije obezbediti da se posle izvršavanja zadatog broja iteracija sačuva niz matrica stanja kroz vreme (stanja sistema u svakoj iteraciji), koji je kompatibilan sa datom funkcijom za animaciju (sledeća ćelija u fajlu).

1. Upotrebom niti koje simuliraju *po jednu ćeliju* i sinhronizacijom Ključevima, Semaforima i Uslovima (7 poena)  
  Svaka nit simulira rad jedne ćelije u sistemu i ima pristup stanjima svojih suseda. Ćelije nemaju pristup globalnom brojaču iteracija, već svaka ćelija interno vodi računa o broju trenutne iteracije. Pored matrice podataka potrebno je uvesti:
  - Listu brojača suseda koji su pročitali trenutnu vrednost (za svaku ćeliju po jedan brojač). Osmi (poslednji) sused koji pročita vrednost budi ćeliju kako bi mogla da upiše novu vrednost u matricu stanja (buđenje realizovati semaforom). Voditi računa o sinhronizaciji suseda koji menjaju vrednost brojača.
  - Uslov (Condition) na kome čekaju sve ćelije koje su upisale novu vrednost u matricu stanja, pre nego što pređu u sledeću iteraciju. 
  - Brojač ćelija, zaštićen ključem, koje su upisale novu vrednost u matricu stanja. Poslednja ćelija aktivira Uslov za sledeću iteraciju. Voditi računa o redosledu uzimanja i puštanja ključa za brojač i ključa za uslov.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import threading
import copy
from threading import Thread
import random
import time
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from multiprocessing import Pool, cpu_count

def animate(steps):
  ''' Prima niz matrica (svaka matrica je stanje u jednom koraku simulacije) 
  prikazuje razvoj sistema'''
  
  def init():
    im.set_data(steps[0])
    return [im]
  
  
  def animate(i):
    im.set_data(steps[i])
    return [im]

  im = plt.matshow(steps[0], interpolation='None', animated=True);
  
  anim = FuncAnimation(im.get_figure(), animate, init_func=init,
                  frames=len(steps), interval=1000, blit=True, repeat=False);
  return anim

In [ ]:
# Generisemo samo jednu random matricu

# Velicina matrice
grid_size = 15
steps = 7

# Generise random matricu date velicine popunjenom nulama i jedinicama
# sa datom verovatnocom (65% za 0 i 35% za 1)
grid_generated = np.array(np.random.choice([0,1], grid_size**2, p=[0.65, 0.35]))

In [ ]:
grid = copy.deepcopy(grid_generated)
grid = grid.reshape(grid_size, grid_size)

# Lista koja ce cuvati istoriju matrica nakon prolaska iteracija
matrix_history = []

zavrsene_celije = 0 # Belezi broj celija koje su zavrsile proveru
zavrsena_celija_lock = threading.Lock()
pristup_susedima_lock = threading.Lock()
sledeca_iteracija = threading.Condition()

class Cell(Thread):

    # Konstruktor za celiju
    def __init__(self, red, kol):
        super().__init__()
        self.red = red
        self.kol = kol
        self.provereni_susedi = np.zeros(8)
        self.iteracija = 0
        self.semafor = threading.Semaphore(0)

        
    # Funkcija koja proverava susede i vraca ukupan broj zivih suseda
    def proveriSusede(self):
        zivi_susedi = 0

        x = np.array([0, 0, -1, 1, -1, -1, 1, 1])
        y = np.array([-1, 1, 0, 0, -1, 1, -1, 1])

        for i in range(0, 8):
          zivi_susedi += grid[(self.red + x[i]) % grid_size, (self.kol + y[i]) % grid_size]

          for t in niti:
            if t.red == (self.red + x[i]) % grid_size and t.kol == (self.kol + y[i]) % grid_size:
              t.provereni_susedi[i] += 1
            t.semafor.release()

        return zivi_susedi

    def update(self, zivi):
      global grid
      global zavrsene_celije

      if grid[self.red, self.kol] == 1:
        if zivi < 2 or zivi > 3:
          grid[self.red, self.kol] = 0
      else:
        if zivi == 3:
          grid[self.red, self.kol] = 1

      zavrsena_celija_lock.acquire()
      zavrsene_celije += 1
      zavrsena_celija_lock.release()

      self.iteracija += 1

    def run(self):
      global steps
      global zavrsene_celije

      for n in range(0, steps):
        zivi_susedi = self.proveriSusede()

        while True:
          self.semafor.acquire()
          pristup_susedima_lock.acquire()
          if np.all(self.provereni_susedi == 1):
            self.provereni_susedi = np.zeros(8)
            pristup_susedima_lock.release()
            break
          pristup_susedima_lock.release()

        sleep_time = random.random()
        time.sleep(sleep_time)

        self.update(zivi_susedi)

        zavrsena_celija_lock.acquire()
        if zavrsene_celije == grid_size**2:
          zavrsene_celije = 0
          zavrsena_celija_lock.release()
          sledeca_iteracija.acquire()
          sledeca_iteracija.notifyAll()
          sledeca_iteracija.release()
          matrix_history.append(grid.copy())
        else:
          zavrsena_celija_lock.release()
          sledeca_iteracija.acquire()
          sledeca_iteracija.wait()
          sledeca_iteracija.release()

niti = []

for i in range(0, grid_size):
  for j in range(0, grid_size):
    nit = Cell(i, j)
    niti.append(nit)

for t in niti:
  t.start()

for t in niti:
  t.join()

anim = animate(matrix_history)
HTML(anim.to_html5_video())

2. Upotrebom niti koje simuliraju *po jednu ćeliju* i sinhronizacijom redovima za poruke (6 poena)  
  Svaka nit simulira rad jedne ćelije u sistemu. Stanje svake ćelije se čuva unutar ćelije (rad sistema se ne oslanja na deljenu matricu stanja). Ćelije podatke o svojem stanju razmenjuju putem reda za poruke. Za potrebe analize rada može se uvesti deljeni niz matrica stanja (i-ti element niza je matrica stanja i-te iteracije), u koji ćelije upisuju svoja stanja (ćelije ne mogu čitati iz ovog niza!).

In [ ]:
import numpy as np
import threading
from threading import Thread, currentThread
from queue import Queue
import sys


grid = copy.deepcopy(grid_generated)
grid = grid.reshape(grid_size, grid_size)

matrix_history = []

class Celija(Thread):

  def __init__(self, red, kol, stanje):
    super().__init__()
    self.red = red
    self.kol = kol
    self.stanje = stanje
    self.queue = Queue()
    self.je_spremno = Queue()
    self.celije_susedi = []
    self.iter = 0

  def inicijalizacija_suseda(self):
    dx = [0, 1, 1, 1, 0, -1, -1, -1]
    dy = [1, 1, 0, -1, -1, -1, 0, 1]
    for i in range(len(dx)):
      for n in niti:
        if n.red == (self.red + dx[i]) % grid_size and n.kol == (self.kol + dy[i]) % grid_size:
          self.celije_susedi.append(n)

  def uzimanje_suseda(self):
    for cs in self.celije_susedi:
      cs.queue.put(self.stanje)
  
  def provera_suseda(self):
    global matrix_history
    zivi = 0
    for i in range(8):
      item = self.queue.get()
      if item == 1:
        zivi += 1
      self.queue.task_done()

    if self.stanje == 1:
      if zivi < 2 or zivi > 3:
        self.stanje = 0
    else:
      if zivi == 3:
        self.stanje = 1
    
    for cs in self.celije_susedi:
      cs.je_spremno.put(1)
    
    for i in range(8):
      self.je_spremno.get()
      self.je_spremno.task_done()
  
  def run(self):
    self.inicijalizacija_suseda()
    for step in range(steps):
      self.iter = step
      self.uzimanje_suseda()
      self.provera_suseda()
      matrix_history[self.iter][self.red][self.kol] = self.stanje
      
  
#      ***main***
niti = []
for i in range(grid_size):
  for j in range(grid_size):
    c = Celija(i,j,grid[i][j])
    niti.append(c)

matr_zeros = np.zeros((grid_size, grid_size))
for _ in range(steps):
  m1 = copy.deepcopy(matr_zeros)
  matrix_history.append(m1)

for n in niti:
  n.start()

for n in niti:
  n.join()

anim = animate(matrix_history)
HTML(anim.to_html5_video())

3. Upotrebom procesa koji simuliraju po jednu ćeliju i sinhronizacijom redovima za poruke (6 poena)
Svaki proces je simulira rad jedne ćelije u sistemu. Stanje svake ćelije se čuva unutar ćelije (rad sistema se ne oslanja na deljenu matricu stanja). Ćelije podatke o svojem stanju razmenjuju putem reda za poruke. Za potrebe analize rada implementirati poseban servis (dodatni proces) kojem sve ćelije javljaju novo stanje prilikom promene (pri čemu poruke sadrže koordinate ćelije, broj iteracije i novo stanje). Servis treba da rekonstruiše i sačuva (ili vrati u glavni program) niz matrica stanja.

In [ ]:
import multiprocessing
import os
import sys
from multiprocessing import Barrier, Queue, Lock


grid = copy.deepcopy(grid_generated)
grid = grid.reshape(grid_size, grid_size)
# Lista koja ce cuvati istoriju matrica nakon prolaska iteracija
matrix_history = []
cells = []


def receive_new_state(grid, queue, mat_queue):
  # elem. iz queue = (row, col, step, new_state)

  matr = np.zeros((grid_size, grid_size))
  for st in range(steps):
    m1 = copy.deepcopy(matr)
    for i in range(grid_size**2):
      elem = queue.get()
      m1[elem[0]][elem[1]] = elem[3]
      if i == grid_size**2 - 1:
        mat_queue.put(m1)
    

class Cell(multiprocessing.Process):
  def __init__(self, row, col, state, glob_q, synq):
    super().__init__()
    self.row = row
    self.col = col

    self.state = state
    self.glob_q = glob_q
    self.synq = synq
    self.queue = Queue()
    self.neighbors = []
    self.iter = 0
  

  def obtain_neighbors(self):
    global cells
    dx = [0, 1, 1, 1, 0, -1, -1, -1]
    dy = [1, 1, 0, -1, -1, -1, 0, 1]

    for i in range(8):
      for c in cells:
        if (self.row + dx[i]) % grid_size == c.row and (self.col + dy[i]) % grid_size == c.col:
          self.neighbors.append(c)
  

  def change_state(self):
    ones = 0
    for c in self.neighbors:
      c.queue.put(self.state)

    for i in range(8):
      num = self.queue.get()
      if num == 1:
        ones += 1

    if self.state == 1:
      if ones < 2 or ones > 3:
        self.state = 0
      else:
        self.state = 1
    else:
      if ones == 3:
        self.state = 1
      else:
        self.state = 0
    
    tup = (self.row, self.col, self.iter, self.state)
    self.glob_q.put(tup)
    

  def run(self):
    self.obtain_neighbors()
    for i in range(steps):
      self.iter = i
      self.change_state()
      self.synq.wait()


if __name__ == '__main__':
  glob_queue = Queue()
  matrix_queue = Queue()
  synq = Barrier(grid_size**2)
  for i in range(grid_size):
    for j in range(grid_size):
      c = Cell(i, j, grid[i][j], glob_queue, synq)
      cells.append(c)
  modify_matrix = multiprocessing.Process(target=receive_new_state, args=(grid, glob_queue, matrix_queue), daemon=True)
  modify_matrix.start()

  for c in cells:
    c.start()

  for c in cells:
    c.join()
  modify_matrix.join()

  while not matrix_queue.empty():
    matrix_history.append(matrix_queue.get())

anim = animate(matrix_history)
HTML(anim.to_html5_video())

4. Upotrebom više procesa kroz *process pool*, generisanjem taskova na nivou skupa ćelija (6 poena)  
  Matricu stanja podeliti na N delova (gde je konfigurabilni parametar) i za svaki deo generisati *task* (poziv funkcije čijim parametrima se definiše koji deo matrice treba obraditi). Funkcija treba da vrati niz koordinata ćelija i njihove nove vrednosti, a matrica za sledeću iteraciju se može kreirati u glavnom programu. Trenutne vrednosti ćelija i suseda se mogu čitati iz deljene matrice.
  

In [ ]:
N = 5 # Konfigurabilni parametar

grid = copy.deepcopy(grid_generated)
grid = grid.reshape(grid_size, grid_size)

def cell_subset_func(task, grid, grid_size):
    koordinate = {}
    
    for x, y in task:
        
        dx = [0, 1, 1, 1, 0, -1, -1, -1]
        dy = [1, 1, 0, -1, -1, -1, 0, 1]
        
        zivi_susedi = 0
        
        for i in range(8):
            zivi_susedi += grid[(x + dx[i]) % grid_size, (y + dy[i]) % grid_size]
            
        if grid[x, y] == 1:
            if zivi_susedi < 2 or zivi_susedi > 3:
                koordinate[(x, y)] = 0
            else:
                koordinate[(x, y)] = 1
        else:
            if zivi_susedi == 3:
                koordinate[(x,y)] = 1
            else:
                koordinate[(x,y)] = 0
    
    return koordinate
    

if __name__ == '__main__':
    matrix_history = [grid.copy()]
    tasks = []
    results = []
    broj_delova = (grid_size**2)//N
    # Dodavanje podskupa celija u posebne taskove
    
    
    cell_subset = []
    counter = 0
    index = 0
    for i in range(0, grid_size):
        for j in range(0, grid_size):
            cell_subset.append((i, j))
            counter += 1
            if counter == broj_delova:
                temp = cell_subset
                tasks.insert(index, temp)
                index += 1
                counter = 0
                cell_subset = []
    
    if len(cell_subset) != 0:
        tasks.insert(index, cell_subset)

    # Process pool
    pool = Pool(cpu_count())
    

    for i in range(steps):
        results = [pool.apply(cell_subset_func, args=(task, grid, grid_size,)) for task in tasks]
        
        for r in results:
            for k, v in r.items():
                grid[k[0], k[1]] = v
                
        matrix_history.append(grid.copy())
    
    pool.close()
    pool.join()

anim = animate(matrix_history)
HTML(anim.to_html5_video())